# DPsim with UDP Logger

This notebook demonstrates using UDP logging for phasor values instead of file logging.

In [1]:
import math
import dpsimpy
from Villas.test_with_docker_compose.udp_logger import DPSimUDPLogger

# DPsim DP simulation
name = 'VILLAS_test_udp'

# Nodes
gnd = dpsimpy.dp.SimNode.gnd
n1 = dpsimpy.dp.SimNode('n1')
n2 = dpsimpy.dp.SimNode('n2')
n3 = dpsimpy.dp.SimNode('n3')

# Initialize node voltages
n1.set_initial_voltage(complex(0, 0) * math.sqrt(2))
n2.set_initial_voltage(complex(0, 0) * math.sqrt(2))

# Components
vs = dpsimpy.dp.ph1.VoltageSource('vs')
vs.set_parameters(V_ref=complex(10000, 0) * math.sqrt(2))

r1 = dpsimpy.dp.ph1.Resistor('r1')
r1.set_parameters(R=1)

l1 = dpsimpy.dp.ph1.Inductor('l1')
l1.set_parameters(L=0.02)

vload = dpsimpy.dp.ph1.VoltageSource('vload')
vload.set_parameters(V_ref=complex(-5000, -1000) * math.sqrt(2))

# Connect components
vs.connect([gnd, n1])
r1.connect([n1, n2])
l1.connect([n2, n3])
vload.connect([n3, gnd])

# Create system topology
system = dpsimpy.SystemTopology(50, [gnd, n1, n2, n3], [vs, r1, l1, vload])

# Create UDP logger instead of file logger
logger = DPSimUDPLogger(name)

# Log node voltages
for i in range(1, len(system.nodes)):
    logger.log_attribute(f'n{i}.v', 'v', system.nodes[i])

# Log component interface currents
logger.log_attribute('r1.i_intf', 'i_intf', r1)
logger.log_attribute('l1.i_intf', 'i_intf', l1)

# Setup simulation
sim = dpsimpy.Simulation(name)
sim.set_domain(dpsimpy.Domain.DP)
sim.set_system(system)
sim.set_time_step(0.00001)
sim.set_final_time(0.1)
sim.add_logger(logger)

print('Starting simulation...')
sim.run()
print('Simulation complete')

# Close the UDP logger
logger.close()

ModuleNotFoundError: No module named 'dpsimpy'